# Midterm 2

## FINM 36700 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is 100 points.
* You have 120 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.


Submission
* You will upload your solution to the `Midterm 2` assignment on Canvas, where you downloaded this. (Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,) and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_2_data.xlsx`

This file has sheets for...
* `info` - names and descriptions of each factor
* `factors (excess returns)` - excess returns on several factors
* `portfolios (excess returns)` - excess returns on industry portfolios
* `risk-free rate` - risk-free rates over time

Note the data is **monthly** so any annualizations should use `12` months in a year.

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 30     |
| 2       | 35     |
| 3       | 20     |
| 4       | 15     |

### Each numbered question is worth 5 points unless otherwise specified.

### Notation
(Hidden LaTeX commands)

$$\newcommand{\betamkt}{\beta^{i,\text{MKT}}}$$
$$\newcommand{\betahml}{\beta^{i,\text{HML}}}$$
$$\newcommand{\betaumd}{\beta^{i,\text{UMD}}}$$
$$\newcommand{\Eri}{E\left[\tilde{r}^{i}\right]}$$
$$\newcommand{\Emkt}{E\left[\tilde{r}^{\text{MKT}}\right]}$$
$$\newcommand{\Ehml}{E\left[\tilde{r}^{\text{HML}}\right]}$$
$$\newcommand{\Eumd}{E\left[\tilde{r}^{\text{UMD}}\right]}$$

# 1. Short Answer

#### No Data Needed

These problems do not require any data file. Rather, analyze them conceptually. 

## 1.

Suppose that we find a set of factors that perfectly hedge any asset. Will these factors work as a linear factor pricing model? 

## 2.

If the Fama-French 3-factor model fit perfectly, would the Treynor ratio be equal for every asset?

## 3.

Suppose the CAPM fits perfectly. Then assets which have higher time-series r-squared metrics on the market factor will have higher Sharpe ratios.

## 4.

Based on the case, what are two ways DFA hopes to generate attractive returns for investors?

## 5.

We analyzed a strategy similar to "AQR's Momentum Funds" (mutual funds.) We found this implementation had much higher returns than the momentum factor of Fama French. What was a major drawback to this construction?

## 6.

From our analysis, momentum has had a negative mean return since 2009. Is this evidence against momentum as a pricing factor? Explain why this is a problem or why it is not a problem.

***

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_2_data.xlsx`

This file has sheets for...
* `info` - names and descriptions of each factor
* `factors (excess returns)` - excess returns on several factors
* `portfolios (excess returns)` - excess returns on industry portfolios
* `risk-free rate` - risk-free rates over time

Note the data is **monthly** so any annualizations should use `12` months in a year.

In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import statsmodels.api as sm
import scipy.stats as stats
import arch
import warnings
warnings.filterwarnings("ignore")
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance
sns.set_theme()
import portfolio_management_helper as pmh

factors = pd.read_excel('../data/midterm_2_data.xlsx', sheet_name = 'factors (excess returns)').set_index('Date')
portfolio = pd.read_excel('../data/midterm_2_data.xlsx', sheet_name = 'portfolios (excess returns)').set_index('Date')

# 2. Linear Factor Pricing Models (LFPMs)

This problem tests the following LFPM:

$$\begin{align}
\Eri = \betamkt \Emkt + \betahml \Ehml + \betaumd \Eumd
\end{align}$$

## 1.

### (8 pts)

Estimate the **time-series (TS)** test of this pricing model. 

For each asset, report the following statistics:
* annualized alpha
* betas
* r-squared

In [152]:
ts = pmh.calc_iterative_regression(
    multiple_y=portfolio,
    X=factors,
    annual_factor=12,
    warnings=False,
    keep_columns=['Annualized Alpha', 'R-Squared','MKT Beta','HML Beta','UMD Beta',]
)
ts

,Annualized Alpha,R-Squared,MKT Beta,HML Beta,UMD Beta
NoDur,0.0293,0.6179,0.7395,0.2046,0.0493
Durbl,0.0107,0.6135,1.2719,0.1736,-0.3200
Manuf,-0.0010,0.8703,1.0495,0.1975,-0.0367
Enrgy,-0.0151,0.4656,0.9922,0.6370,0.0752
HiTec,0.0282,0.8295,1.1550,-0.6371,-0.1406
Telcm,0.0035,0.5881,0.8373,0.0944,-0.0845
Shops,0.0267,0.7422,0.9469,-0.0422,-0.0150
Hlth,0.0319,0.5805,0.7576,-0.1199,0.0741
Utils,0.0137,0.3427,0.5279,0.3530,0.1086
Other,-0.0198,0.9101,1.1154,0.4268,-0.0487


## 2.

### (7pts)

Estimate the **cross-sectional (CS)** test of the pricing model. 

Include an intercept in your cross-sectional test.

Report the
* annualized intercept
* annualized regression coefficients
* r-squared

In [153]:
cs = pmh.calc_cross_section_regression(
    portfolio,
    factors,
    annual_factor=12,
    provided_excess_returns=True,
)


Lambda represents the premium calculated by the cross-section regression and the historical premium is the average of the factor excess returns


In [154]:
cs[["Annualized Eta","MKT Annualized Lambda","HML Annualized Lambda","UMD Annualized Lambda","R-Squared"]]

,Annualized Eta,MKT Annualized Lambda,HML Annualized Lambda,UMD Annualized Lambda,R-Squared
MKT + HML + UMD Cross-Section Regression,0.0637,0.0320,-0.0158,0.0303,0.3662


## 3.

Report the annualized factor premia (expected excess returns of the three factors) as implied by each of the TS and CS estimations.

In [155]:
#time series
pmh.calc_cross_section_regression(
    portfolio,
    factors,
    annual_factor=12,
    provided_excess_returns=True,
    keep_columns=["MKT Annualized Historical Premium","HML Annualized Historical Premium","UMD Annualized Historical Premium"],
    return_historical_premium = True,
    return_annualized_premium = True,
)

Lambda represents the premium calculated by the cross-section regression and the historical premium is the average of the factor excess returns


,MKT Annualized Historical Premium,HML Annualized Historical Premium,UMD Annualized Historical Premium
MKT + HML + UMD Cross-Section Regression,0.0839,0.0250,0.0617


In [156]:
#cross section
pmh.calc_cross_section_regression(
    portfolio,
    factors,
    annual_factor=12,
    provided_excess_returns=True,
    keep_columns=["MKT Annualized Lambda","HML Annualized Lambda","UMD Annualized Lambda"],
    # return_historical_premium = True,
    return_annualized_premium = True,
    # compare_premiums = True,
)

Lambda represents the premium calculated by the cross-section regression and the historical premium is the average of the factor excess returns


,MKT Annualized Lambda,HML Annualized Lambda,UMD Annualized Lambda
MKT + HML + UMD Cross-Section Regression,0.0320,-0.0158,0.0303


## 4.

Use the r-squared statistics from the TS and CS tests above to assess whether these factors are effective for decomposition and/or pricing.

Be specific as to how the r-squared statistics from the TS and CS tests impact your conclusions.

## 5.

Report the annualized pricing mean absolute error (MAE) implied by each of the TS and CS estimations.

In [157]:
cs[["TS Annualized MAE","CS Annualized MAE"]]

,TS Annualized MAE,CS Annualized MAE
MKT + HML + UMD Cross-Section Regression,0.0180,0.0079


## 6.

Which asset has the highest premium as implied by the TS estimation? And as implied by the CS estimation? (For the latter, feel free to include the cross-sectional intercept.)

In [158]:
#TS
ts = ts.rename(columns={'MKT Beta':'MKT','HML Beta':'HML','UMD Beta':'UMD'})
((factors.mean()*12)[['MKT','HML','UMD']] * ts[['MKT','HML','UMD']]).sum(axis =1).to_frame('TS Predicted Premium').nlargest(1, 'TS Predicted Premium')

,TS Predicted Premium
Enrgy,0.1038


In [159]:
#CS
cs = cs.rename(columns= {'MKT Annualized Lambda':'MKT','HML Annualized Lambda':'HML',
                        'UMD Annualized Lambda':'UMD'})
(ts[['MKT','HML','UMD']] * cs[['MKT','HML','UMD']].values[0]).sum(axis=1).nlargest(1).to_frame('CS Predicted Premium without intercept')

,CS Predicted Premium without intercept
HiTec,0.0427


In [160]:
predicted_cs = cs['Annualized Eta'][0] + (ts[['MKT','HML','UMD']] * cs[['MKT','HML','UMD']].values[0]).sum(axis=1)
(predicted_cs).nlargest(1).to_frame('CS Predicted Premium with intercept')

,CS Predicted Premium with intercept
HiTec,0.1065


***

# 3. Additional Analysis

## 1. 

Consider the three-factor pricing model above. How can we assess whether all three factors are useful in this pricing model? 

Specifically, discuss whether the previously estimated regression betas would be informative. If not, what other statistic could we calculate?

## 2.

Suppose we are testing the 3-factor model above, and now we want to allow for time-varying betas.

How could we test the model while allowing for this?

Be specific about the number of regressions we would run and the nature of these regressions.

## 3.

State one advantage and one disadvantage of using the CS estimation as opposed to the TS estimation in fitting the LFPM to the data.

## 4.

Suppose we are investing in just the assets included in our data set. We want to implement a momentum strategy.

Relative to the momentum strategies we studied, do you expect this strategy would have higher or lower...
* mean
* volatility

Explain.

***

# 4. Returns Over Time

## 1.

If Barnstable’s assumptions hold, (log iid returns, normally distributed,) then in what sense is an investment safer in the long-run? And in what sense is it riskier in the long-run?

## 2. 

### (10pts)

Data 
* Make use of the `risk-free rate` tab.
* Construct the **total** factor returns by adding the risk-free rate to the excess `MKT` and `HML` factor returns.

Assumptions
* The total returns are lognormally distributed and iid. 

Report the probability that `MKT` will outperform `HML` over the following 5 years.

***